Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic regression with L2 regularization (simple gradient descent)

In [7]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(weights)
  loss = loss + 0.5 * beta * regularizers
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 35.337681
Training accuracy: 9.4%
Validation accuracy: 13.5%
Loss at step 100: 10.541477
Training accuracy: 72.2%
Validation accuracy: 70.6%
Loss at step 200: 6.219317
Training accuracy: 76.1%
Validation accuracy: 73.9%
Loss at step 300: 3.807533
Training accuracy: 78.7%
Validation accuracy: 76.1%
Loss at step 400: 2.443305
Training accuracy: 81.0%
Validation accuracy: 77.7%
Loss at step 500: 1.670501
Training accuracy: 82.6%
Validation accuracy: 79.1%
Loss at step 600: 1.230737
Training accuracy: 83.6%
Validation accuracy: 80.4%
Loss at step 700: 0.978719
Training accuracy: 84.3%
Validation accuracy: 81.0%
Loss at step 800: 0.833275
Training accuracy: 84.6%
Validation accuracy: 81.3%
Test accuracy: 88.5%


Test accuracy w/o L2 regularization: 82.8%

### Logistic regression with L2 regularization (stochastic gradient descent)

In [9]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  # Loss function using L2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(weights)
  loss = loss + 0.5 * beta * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 31.121889
Minibatch accuracy: 14.1%
Validation accuracy: 16.8%
Minibatch loss at step 500: 1.681813
Minibatch accuracy: 81.2%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 0.874245
Minibatch accuracy: 80.5%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 0.694744
Minibatch accuracy: 83.6%
Validation accuracy: 82.5%
Minibatch loss at step 2000: 0.707785
Minibatch accuracy: 81.2%
Validation accuracy: 82.3%
Minibatch loss at step 2500: 0.688534
Minibatch accuracy: 81.2%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 0.927086
Minibatch accuracy: 78.1%
Validation accuracy: 81.5%
Test accuracy: 88.0%


Test accuracy w/o L2 regularization: 85.7%

### Neural network with L2 regularization

In [13]:
batch_size = 128
num_hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_01 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    weights_12 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases_01 = tf.Variable(tf.zeros([num_hidden_nodes]))
    biases_12 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    z_01 = tf.matmul(tf_train_dataset, weights_01) + biases_01
    h1 = tf.nn.relu(z_01)
    z_12 = tf.matmul(h1, weights_12) + biases_12
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(z_12, tf_train_labels))
    regularizers = tf.nn.l2_loss(weights_12) + tf.nn.l2_loss(biases_12) + tf.nn.l2_loss(weights_01) + tf.nn.l2_loss(biases_01)
    loss = loss + 0.5 * beta * regularizers
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(z_12)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_01) + biases_01)
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_relu, weights_12) + biases_12) 
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, weights_01) + biases_01)
    test_prediction = tf.nn.softmax(
        tf.matmul(test_relu, weights_12) + biases_12)

In [14]:
num_steps = 3001

with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1923.689087
Minibatch accuracy: 11.7%
Validation accuracy: 37.9%
Minibatch loss at step 500: 127.290527
Minibatch accuracy: 87.5%
Validation accuracy: 82.9%
Minibatch loss at step 1000: 10.974431
Minibatch accuracy: 86.7%
Validation accuracy: 86.1%
Minibatch loss at step 1500: 1.476885
Minibatch accuracy: 83.6%
Validation accuracy: 85.7%
Minibatch loss at step 2000: 0.703694
Minibatch accuracy: 84.4%
Validation accuracy: 85.8%
Minibatch loss at step 2500: 0.661868
Minibatch accuracy: 85.2%
Validation accuracy: 85.3%
Minibatch loss at step 3000: 0.775606
Minibatch accuracy: 82.0%
Validation accuracy: 85.4%
Test accuracy: 91.0%


Test accuracy w/o L2 regularization: 88.6%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

### Logistic regression with L2 regularization (simple gradient descent) and small training data

In [15]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(weights)
  loss = loss + 0.5 * beta * regularizers
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [16]:
num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 32.668457
Training accuracy: 10.0%
Validation accuracy: 13.9%
Loss at step 100: 9.197253
Training accuracy: 100.0%
Validation accuracy: 51.2%
Loss at step 200: 5.592769
Training accuracy: 100.0%
Validation accuracy: 54.5%
Loss at step 300: 3.410747
Training accuracy: 100.0%
Validation accuracy: 57.8%
Loss at step 400: 2.089845
Training accuracy: 100.0%
Validation accuracy: 61.1%
Loss at step 500: 1.290649
Training accuracy: 100.0%
Validation accuracy: 64.1%
Loss at step 600: 0.807440
Training accuracy: 100.0%
Validation accuracy: 66.5%
Loss at step 700: 0.515451
Training accuracy: 100.0%
Validation accuracy: 68.3%
Loss at step 800: 0.339054
Training accuracy: 100.0%
Validation accuracy: 69.5%
Test accuracy: 75.6%


Training accuracy goes to 100% due to the small sample size, but then the validation accuracy is worst due to overfitting.

### Logistic regression with L2 regularization (stochastic gradient descent) and small training data

In [17]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  # Loss function using L2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(weights)
  loss = loss + 0.5 * beta * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [18]:
num_steps = 3001

train_dataset_subset = train_dataset[:500, :]
train_labels_subset = train_labels[:500]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_subset.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_subset[offset:(offset + batch_size), :]
    batch_labels = train_labels_subset[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 33.725357
Minibatch accuracy: 3.9%
Validation accuracy: 8.7%
Minibatch loss at step 500: 1.371678
Minibatch accuracy: 98.4%
Validation accuracy: 71.3%
Minibatch loss at step 1000: 0.305689
Minibatch accuracy: 100.0%
Validation accuracy: 74.9%
Minibatch loss at step 1500: 0.206317
Minibatch accuracy: 100.0%
Validation accuracy: 75.1%
Minibatch loss at step 2000: 0.197743
Minibatch accuracy: 100.0%
Validation accuracy: 75.0%
Minibatch loss at step 2500: 0.194401
Minibatch accuracy: 100.0%
Validation accuracy: 75.0%
Minibatch loss at step 3000: 0.194328
Minibatch accuracy: 100.0%
Validation accuracy: 75.2%
Test accuracy: 81.7%


Minibatch accuracy nearly to 100% due to the small sample size, but then the validation accuracy is worst due to overfitting.

### Neural network with L2 regularization and small training data

In [19]:
batch_size = 128
num_hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                   shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_01 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    weights_12 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases_01 = tf.Variable(tf.zeros([num_hidden_nodes]))
    biases_12 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    z_01 = tf.matmul(tf_train_dataset, weights_01) + biases_01
    h1 = tf.nn.relu(z_01)
    z_12 = tf.matmul(h1, weights_12) + biases_12
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(z_12, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights_12) + tf.nn.l2_loss(biases_12) +
                    tf.nn.l2_loss(weights_01) + tf.nn.l2_loss(biases_01))
    loss = loss + 0.5 * beta * regularizers
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(z_12)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_01) + biases_01)
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_relu, weights_12) + biases_12) 
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, weights_01) + biases_01)
    test_prediction = tf.nn.softmax(
        tf.matmul(test_relu, weights_12) + biases_12)

In [20]:
num_steps = 3001

train_dataset_subset = train_dataset[:500, :]
train_labels_subset = train_labels[:500]

with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_subset.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_subset[offset:(offset + batch_size), :]
        batch_labels = train_labels_subset[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1888.007812
Minibatch accuracy: 10.2%
Validation accuracy: 30.8%
Minibatch loss at step 500: 128.730591
Minibatch accuracy: 100.0%
Validation accuracy: 75.0%
Minibatch loss at step 1000: 10.601433
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 1500: 1.026079
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 0.243927
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 2500: 0.173688
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 3000: 0.165460
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Test accuracy: 84.0%


Minibatch accuracy goes to 100% due to the small sample size, but then the validation accuracy is worst due to overfitting.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

### Neural network with L2 regularization and dropout

In [21]:
batch_size = 128
num_hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_01 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    weights_12 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases_01 = tf.Variable(tf.zeros([num_hidden_nodes]))
    biases_12 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    z_01 = tf.matmul(tf_train_dataset, weights_01) + biases_01
    h1 = tf.nn.relu(z_01)
    
    # Dropout on hidden layer.
    keep_prob = tf.placeholder(tf.float32)
    h1_drop = tf.nn.dropout(h1, keep_prob)
    z_12 = tf.matmul(h1_drop, weights_12) + biases_12
       
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(z_12, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights_12) + tf.nn.l2_loss(biases_12) +
                    tf.nn.l2_loss(weights_01) + tf.nn.l2_loss(biases_01))
    loss = loss + 0.5 * beta * regularizers
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(z_12)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_01) + biases_01)
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_relu, weights_12) + biases_12) 
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, weights_01) + biases_01)
    test_prediction = tf.nn.softmax(
        tf.matmul(test_relu, weights_12) + biases_12)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2000.961060
Minibatch accuracy: 12.5%
Validation accuracy: 34.9%
Minibatch loss at step 500: 129.762817
Minibatch accuracy: 76.6%
Validation accuracy: 82.6%
Minibatch loss at step 1000: 11.061076
Minibatch accuracy: 82.0%
Validation accuracy: 85.0%
Minibatch loss at step 1500: 1.519651
Minibatch accuracy: 83.6%
Validation accuracy: 85.3%
Minibatch loss at step 2000: 0.800788
Minibatch accuracy: 82.0%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 0.698235
Minibatch accuracy: 83.6%
Validation accuracy: 84.3%
Minibatch loss at step 3000: 0.893140
Minibatch accuracy: 78.1%
Validation accuracy: 83.8%
Test accuracy: 90.0%


Test accuracy w/o Dropout: 91.0%

### Neural network with L2 regularization, dropout and small dataset

In [23]:
num_steps = 3001

train_dataset_subset = train_dataset[:500, :]
train_labels_subset = train_labels[:500]

with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_subset.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_subset[offset:(offset + batch_size), :]
        batch_labels = train_labels_subset[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2010.735474
Minibatch accuracy: 11.7%
Validation accuracy: 43.5%
Minibatch loss at step 500: 129.178909
Minibatch accuracy: 98.4%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 10.606963
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 1500: 1.032677
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 2000: 0.250695
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 0.192443
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 3000: 0.174683
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Test accuracy: 84.2%


Test accuracy w/o Dropout: 84.0%

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


To beat: 90.0%

### Neural network with L2 regularization and dropout, smaller beta and bigger num steps

In [24]:
batch_size = 128
num_hidden_nodes = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_01 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    weights_12 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases_01 = tf.Variable(tf.zeros([num_hidden_nodes]))
    biases_12 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    z_01 = tf.matmul(tf_train_dataset, weights_01) + biases_01
    h1 = tf.nn.relu(z_01)
    
    # Dropout on hidden layer.
    keep_prob = tf.placeholder(tf.float32)
    h1_drop = tf.nn.dropout(h1, keep_prob)
    z_12 = tf.matmul(h1_drop, weights_12) + biases_12
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(z_12, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights_12) + tf.nn.l2_loss(biases_12) +
                    tf.nn.l2_loss(weights_01) + tf.nn.l2_loss(biases_01))
    loss = loss + 0.5 * beta * regularizers
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(z_12)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_01) + biases_01)
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_relu, weights_12) + biases_12) 
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, weights_01) + biases_01)
    test_prediction = tf.nn.softmax(
        tf.matmul(test_relu, weights_12) + biases_12)

In [25]:
num_steps = 6001

with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 608.380920
Minibatch accuracy: 7.0%
Validation accuracy: 40.2%
Minibatch loss at step 500: 142.801056
Minibatch accuracy: 70.3%
Validation accuracy: 82.1%
Minibatch loss at step 1000: 103.112289
Minibatch accuracy: 75.8%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 79.679962
Minibatch accuracy: 73.4%
Validation accuracy: 79.5%
Minibatch loss at step 2000: 58.763531
Minibatch accuracy: 75.8%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 46.355400
Minibatch accuracy: 76.6%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 36.277657
Minibatch accuracy: 72.7%
Validation accuracy: 83.1%
Minibatch loss at step 3500: 28.342201
Minibatch accuracy: 74.2%
Validation accuracy: 84.1%
Minibatch loss at step 4000: 22.244642
Minibatch accuracy: 82.0%
Validation accuracy: 84.6%
Minibatch loss at step 4500: 17.197151
Minibatch accuracy: 72.7%
Validation accuracy: 84.8%
Minibatch loss at step 5000: 13.154745
Minibatch accuracy: 80.5%
Valid

* beta 0.001, num_steps 6001 : 92.7%

### Neural network with L2 regularization and dropout, multiple keep prob

In [28]:
batch_size = 128
num_hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_01 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    weights_12 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases_01 = tf.Variable(tf.zeros([num_hidden_nodes]))
    biases_12 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    z_01 = tf.matmul(tf_train_dataset, weights_01) + biases_01
    h1 = tf.nn.relu(z_01)
    
    # Dropout on hidden layer.
    keep_prob = tf.placeholder(tf.float32)
    h1_drop = tf.nn.dropout(h1, keep_prob)
    z_12 = tf.matmul(h1_drop, weights_12) + biases_12

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(z_12, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights_12) + tf.nn.l2_loss(biases_12) +
                    tf.nn.l2_loss(weights_01) + tf.nn.l2_loss(biases_01))
    loss = loss + 0.5 * beta * regularizers
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(z_12)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_01) + biases_01)
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_relu, weights_12) + biases_12) 
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, weights_01) + biases_01)
    test_prediction = tf.nn.softmax(
        tf.matmul(test_relu, weights_12) + biases_12)

In [29]:
num_steps = 3001

for kp in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    print("Keep prob: " + str(kp))
    with tf.Session(graph=graph) as session:
          tf.initialize_all_variables().run()
          print("Initialized")
          for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
          print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Keep prob: 0.5
Initialized
Minibatch loss at step 0: 2053.375488
Minibatch accuracy: 6.2%
Validation accuracy: 34.2%
Minibatch loss at step 500: 129.343460
Minibatch accuracy: 78.9%
Validation accuracy: 82.3%
Minibatch loss at step 1000: 11.090308
Minibatch accuracy: 82.0%
Validation accuracy: 85.0%
Minibatch loss at step 1500: 1.575262
Minibatch accuracy: 82.0%
Validation accuracy: 85.0%
Minibatch loss at step 2000: 0.777193
Minibatch accuracy: 83.6%
Validation accuracy: 85.1%
Minibatch loss at step 2500: 0.730821
Minibatch accuracy: 82.0%
Validation accuracy: 84.3%
Minibatch loss at step 3000: 0.913440
Minibatch accuracy: 77.3%
Validation accuracy: 83.8%
Test accuracy: 89.8%
Keep prob: 0.6
Initialized
Minibatch loss at step 0: 2010.023926
Minibatch accuracy: 10.2%
Validation accuracy: 35.3%
Minibatch loss at step 500: 128.865845
Minibatch accuracy: 78.9%
Validation accuracy: 82.2%
Minibatch loss at step 1000: 11.033095
Minibatch accuracy: 85.2%
Validation accuracy: 85.4%
Minibatch lo

* keep_prob 1.0 : 91.1%

### Neural network with L2 regularization and dropout, multiple learning rate


In [30]:
batch_size = 128
num_hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_01 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    weights_12 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases_01 = tf.Variable(tf.zeros([num_hidden_nodes]))
    biases_12 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    z_01 = tf.matmul(tf_train_dataset, weights_01) + biases_01
    h1 = tf.nn.relu(z_01)
    
    # Dropout on hidden layer.
    keep_prob = tf.placeholder(tf.float32)
    h1_drop = tf.nn.dropout(h1, keep_prob)
    z_12 = tf.matmul(h1_drop, weights_12) + biases_12
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(z_12, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights_12) + tf.nn.l2_loss(biases_12) +
                    tf.nn.l2_loss(weights_01) + tf.nn.l2_loss(biases_01))
    loss = loss + 0.5 * beta * regularizers
  
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learn_rate = tf.placeholder(tf.float32)
    learning_rate = tf.train.exponential_decay(learn_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(z_12)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_01) + biases_01)
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_relu, weights_12) + biases_12) 
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, weights_01) + biases_01)
    test_prediction = tf.nn.softmax(
        tf.matmul(test_relu, weights_12) + biases_12)

In [31]:
num_steps = 3001

for lr in np.arange(0.001, 0.010, 0.001).tolist():
    print("Learning rate: " + str(lr))
    with tf.Session(graph=graph) as session:
          tf.initialize_all_variables().run()
          print("Initialized")
          for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5, learn_rate : lr}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
          print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Learning rate: 0.001
Initialized
Minibatch loss at step 0: 2094.931641
Minibatch accuracy: 7.0%
Validation accuracy: 5.5%
Minibatch loss at step 500: 1753.029297
Minibatch accuracy: 42.2%
Validation accuracy: 65.7%
Minibatch loss at step 1000: 1686.015015
Minibatch accuracy: 59.4%
Validation accuracy: 73.9%
Minibatch loss at step 1500: 1662.086914
Minibatch accuracy: 64.1%
Validation accuracy: 76.2%
Minibatch loss at step 2000: 1655.341553
Minibatch accuracy: 68.0%
Validation accuracy: 77.4%
Minibatch loss at step 2500: 1621.012329
Minibatch accuracy: 69.5%
Validation accuracy: 78.3%
Minibatch loss at step 3000: 1637.609985
Minibatch accuracy: 63.3%
Validation accuracy: 78.9%
Test accuracy: 86.4%
Learning rate: 0.002
Initialized
Minibatch loss at step 0: 2074.706299
Minibatch accuracy: 7.0%
Validation accuracy: 9.6%
Minibatch loss at step 500: 1700.015625
Minibatch accuracy: 50.0%
Validation accuracy: 74.2%
Minibatch loss at step 1000: 1652.545532
Minibatch accuracy: 60.9%
Validation a

* learn_rate 0.004: 89.2%

### Neural network with L2 regularization and dropout, learning rate and 2 layers

In [36]:
batch_size = 128
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 512
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_01 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes1]))
    weights_12 = tf.Variable(tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2]))
    weights_23 = tf.Variable(tf.truncated_normal([num_hidden_nodes2, num_labels]))
    biases_01 = tf.Variable(tf.zeros([num_hidden_nodes1]))
    biases_12 = tf.Variable(tf.zeros([num_hidden_nodes2]))
    biases_23 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    keep_prob = tf.placeholder(tf.float32)
    z_01 = tf.matmul(tf_train_dataset, weights_01) + biases_01
    h1 = tf.nn.relu(z_01)
    h1_drop = tf.nn.dropout(h1, keep_prob)
    z_12 = tf.matmul(h1_drop, weights_12) + biases_12
    h2 = tf.nn.relu(z_12)
    h2_drop = tf.nn.dropout(h2, keep_prob)
    z_23 = tf.matmul(h2_drop, weights_23) + biases_23
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(z_23, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights_23) + tf.nn.l2_loss(biases_23) +
                    tf.nn.l2_loss(weights_12) + tf.nn.l2_loss(biases_12) +
                    tf.nn.l2_loss(weights_01) + tf.nn.l2_loss(biases_01))
    loss = loss + 0.5 * beta * regularizers
  
    # Optimizer.
    learn_rate = tf.placeholder(tf.float32)
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learn_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(z_23)
    valid_relu_01 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_01) + biases_01)
    valid_relu_12 = tf.nn.relu(tf.matmul(valid_relu_01, weights_12) + biases_12)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu_12, weights_23) + biases_23) 
    test_relu_01 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_01) + biases_01)
    test_relu_12 = tf.nn.relu(tf.matmul(test_relu_01, weights_12) + biases_12)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu_12, weights_23) + biases_23)

In [37]:
num_steps = 3001

with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                     keep_prob : 0.5, learn_rate : 0.004}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 12297.138672
Minibatch accuracy: 9.4%
Validation accuracy: 18.1%
Minibatch loss at step 500: 2830.758301
Minibatch accuracy: 62.5%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 2567.862549
Minibatch accuracy: 67.2%
Validation accuracy: 77.2%
Minibatch loss at step 1500: 2455.312012
Minibatch accuracy: 64.8%
Validation accuracy: 73.9%
Minibatch loss at step 2000: 2382.784424
Minibatch accuracy: 69.5%
Validation accuracy: 75.2%
Minibatch loss at step 2500: 2342.846436
Minibatch accuracy: 59.4%
Validation accuracy: 76.0%
Minibatch loss at step 3000: 2285.843262
Minibatch accuracy: 60.9%
Validation accuracy: 75.8%
Test accuracy: 82.7%


### Neural network with L2 regularization and dropout, 2 layers, optimized beta, optimized num_steps, optimized keep prob and optimized learning rate

In [38]:
batch_size = 128
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 512
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_01 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes1]))
    weights_12 = tf.Variable(tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2]))
    weights_23 = tf.Variable(tf.truncated_normal([num_hidden_nodes2, num_labels]))
    biases_01 = tf.Variable(tf.zeros([num_hidden_nodes1]))
    biases_12 = tf.Variable(tf.zeros([num_hidden_nodes2]))
    biases_23 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    keep_prob = tf.placeholder(tf.float32)
    z_01 = tf.matmul(tf_train_dataset, weights_01) + biases_01
    h1 = tf.nn.relu(z_01)
    h1_drop = tf.nn.dropout(h1, keep_prob)
    z_12 = tf.matmul(h1_drop, weights_12) + biases_12
    h2 = tf.nn.relu(z_12)
    h2_drop = tf.nn.dropout(h2, keep_prob)
    z_23 = tf.matmul(h2_drop, weights_23) + biases_23

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(z_23, tf_train_labels))
    regularizers = (tf.nn.l2_loss(weights_23) + tf.nn.l2_loss(biases_23) +
                    tf.nn.l2_loss(weights_12) + tf.nn.l2_loss(biases_12) +
                    tf.nn.l2_loss(weights_01) + tf.nn.l2_loss(biases_01))
    loss = loss + 0.5 * beta * regularizers
  
    # Optimizer.
    learn_rate = tf.placeholder(tf.float32)
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learn_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(z_23)
    valid_relu_01 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_01) + biases_01)
    valid_relu_12 = tf.nn.relu(tf.matmul(valid_relu_01, weights_12) + biases_12)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu_12, weights_23) + biases_23) 
    test_relu_01 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_01) + biases_01)
    test_relu_12 = tf.nn.relu(tf.matmul(test_relu_01, weights_12) + biases_12)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu_12, weights_23) + biases_23)

In [39]:
num_steps = 6001

with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                     keep_prob : 1.0, learn_rate : 0.004}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6746.208008
Minibatch accuracy: 6.2%
Validation accuracy: 12.7%
Minibatch loss at step 500: 456.937256
Minibatch accuracy: 79.7%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 370.144928
Minibatch accuracy: 75.8%
Validation accuracy: 79.0%
Minibatch loss at step 1500: 305.491821
Minibatch accuracy: 84.4%
Validation accuracy: 78.7%
Minibatch loss at step 2000: 282.505951
Minibatch accuracy: 75.8%
Validation accuracy: 78.0%
Minibatch loss at step 2500: 280.822235
Minibatch accuracy: 78.1%
Validation accuracy: 77.2%
Minibatch loss at step 3000: 276.287720
Minibatch accuracy: 70.3%
Validation accuracy: 75.9%
Minibatch loss at step 3500: 284.406555
Minibatch accuracy: 71.9%
Validation accuracy: 75.3%
Minibatch loss at step 4000: 267.906647
Minibatch accuracy: 78.1%
Validation accuracy: 74.7%
Minibatch loss at step 4500: 259.273651
Minibatch accuracy: 71.1%
Validation accuracy: 74.8%
Minibatch loss at step 5000: 265.175323
Minibatch accuracy: 68